Librerias

In [ ]:
!pip install pandas numpy scikit-learn

In [8]:
# importamos la librerias

import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree


In [10]:
# Cargar el archivo CSV (usar latin1 para evitar errores con acentos)
df = pd.read_csv("C:/Users/hgarb/OneDrive/Escritorio/Portfolio/01_coordenadas_balltree_folium/DS_Mantenimientos_ENE_JUL.csv", encoding="latin1")


In [16]:
df.head()

,Fecha transito,Fecha sitio,Mes,Latitude,Longitude,Costo Unitario,Unnamed: 8
0,14/01/2025 12:30,14/01/2025,1,19.700180,-99.148723,350,NaN
1,18/02/2025 13:12,18/02/2025,2,19.700176,-99.148729,800,NaN
2,11/02/2025 16:07,11/02/2025,2,19.677593,-99.097065,350,NaN
3,10/01/2025 10:56,10/01/2025,1,19.677593,-99.097065,350,NaN
4,04/01/2025 14:53,04/01/2025,1,19.677393,-99.133952,350,NaN


In [18]:
df.columns = [c.strip() for c in df.columns]
df = df.dropna(axis=1, how='all')

In [19]:
df.head()

,Fecha transito,Fecha sitio,Mes,Latitude,Longitude,Costo Unitario,Unnamed: 8
0,14/01/2025 12:30,14/01/2025,1,19.700180,-99.148723,350,NaN
1,18/02/2025 13:12,18/02/2025,2,19.700176,-99.148729,800,NaN
2,11/02/2025 16:07,11/02/2025,2,19.677593,-99.097065,350,NaN
3,10/01/2025 10:56,10/01/2025,1,19.677593,-99.097065,350,NaN
4,04/01/2025 14:53,04/01/2025,1,19.677393,-99.133952,350,NaN


In [22]:
df = df.drop(columns=['Unnamed: 8'])

In [23]:
df.head()

,Fecha transito,Fecha sitio,Mes,Latitude,Longitude,Costo Unitario
0,14/01/2025 12:30,14/01/2025,1,19.700180,-99.148723,350
1,18/02/2025 13:12,18/02/2025,2,19.700176,-99.148729,800
2,11/02/2025 16:07,11/02/2025,2,19.677593,-99.097065,350
3,10/01/2025 10:56,10/01/2025,1,19.677593,-99.097065,350
4,04/01/2025 14:53,04/01/2025,1,19.677393,-99.133952,350


In [30]:

# =========================
# 1) Agrupar por cercanía (4 m) con BallTree
# =========================
coords = np.radians(df[["Latitude", "Longitude"]].values)
tree = BallTree(coords, metric='haversine')

radius = 4 / 6371000.0  # 4 metros a radianes

group_ids = [-1] * len(df)
current_group = 0

for i in range(len(df)):
    if group_ids[i] != -1:
        continue
    idx = tree.query_radius([coords[i]], r=radius)[0]
    for j in idx:
        group_ids[j] = current_group
    current_group += 1

df["GrupoID"] = group_ids

# =========================
# 2) Detectar grupos repetidos por Mes
# =========================
repetidos = (
    df.groupby("GrupoID")["Mes"]
      .nunique()
      .reset_index()
      .rename(columns={"Mes": "Meses_unicos"})
)
repetidos = repetidos[repetidos["Meses_unicos"] > 1]

resultado = df[df["GrupoID"].isin(repetidos["GrupoID"])].copy()

# =========================
# 3) Exportar resultado a Excel (única hoja)
# =========================
nombre_archivo = "resultado.xlsx"
resultado.to_excel(nombre_archivo, index=False)

print(f"✅ Archivo '{nombre_archivo}' creado con los datos filtrados.")

✅ Archivo 'resultado.xlsx' creado con los datos filtrados.
